In [3]:
import pandas as pd
import html5lib

ImportError: No module named html5lib

In [2]:
pd.read_html(r'http://modis.higp.hawaii.edu/cgi-bin/mergeimage?maptype=alerts&jyear=2019&jday=23&jperiod=1&lonmin=-140&latmin=33&lonmax=-95&latmax=78')

ImportError: html5lib not found, please install it